In [1]:
from dotenv import load_dotenv, dotenv_values

from selenium import webdriver
from selenium.webdriver.firefox.options import Options
from time import sleep
#from parsel import Selector
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from facebook_scraper import get_posts
from multiprocessing import Pool, cpu_count
import re
from requests import get
import pandas as pd
import numpy as np
import time

load_dotenv()

True

## **SETUP**

In [2]:
config = dotenv_values("../.env")

## **Définiton de fonctions**

In [15]:
# initialisation de selemium
def recherche_google(localite,nbrePage):
    #chargement du navigateur et de webdriver
    driver = webdriver.Firefox()
    driver.get("https://www.google.com")
    search_query = driver.find_element(By.NAME,'q')
    search_query.send_keys('site:facebook.com AND'+'"'+localite + '"AND "' + "maison" + '"')
    search_query.send_keys(Keys.RETURN)
    #recherche des liens
    
    lienBon=[]
    n=0
    test=True
    while test:
        # faire une pause de 3 secondes
        time.sleep(3)
        if n< nbrePage:
            try:
                facebook_urls  = driver.find_elements(By.XPATH,"//*[@class='yuRUbf']/a")
                print(facebook_urls)
                for url in facebook_urls:
                    lienBon.append(url.get_attribute('href'))

                suivantpage=driver.find_element(By.ID,'pnnext')
                print(suivantpage)
                time.sleep(1)
                suivantpage.click()
                n=n+1
            except Exception as e:
                print(e)
                test=False   
        else:
            test=False
    return lienBon

def postGroup(nbpage,idGroup,nMax):
    n=0
    ListPost=[]
    for post in get_posts(idGroup, cookies='/Users/turing/Desktop/turing/projets/loyer_scrapping/res/cookies.txt', extra_info=True, pages=nbpage, options={"comments": False}):
        print(post)
        ListPost.append(post)
        if n==nMax:
            break
    return ListPost

def multiprocessing(postGroup , list):
    with Pool(cpu_count()) as p:
        print(cpu_count())
        rec=p.starmap( postGroup,list)    
        p.terminate()
        p.join()
        return rec
# Lanceur du script

In [ ]:
ListIdGroup=recherche_google("Yopougon",3)

In [ ]:
if __name__ == '__main__':
        start_time = time.time()
        list = ListIdGroup
        ListeDictionnaire=multiprocessing(postGroup , list)
        # decomenter la ligne suivante pour afficher les valeurs
        print(ListeDictionnaire)
        pd.DataFrame(ListeDictionnaire).to_csv("../data/output.csv")
        end_time = time.time() - start_time
        # Fin des traitement
        print("Temps total de compilation "+str(end_time))